In [1]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import missingno as msno

# Load Oil datasets
df = pd.read_csv('/content/oil_updated_dataset.csv')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 575 entries, 0 to 574
Data columns (total 59 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   typeCode                  575 non-null    object 
 1   freqCode                  575 non-null    object 
 2   refPeriodId               575 non-null    int64  
 3   Year                      575 non-null    int64  
 4   refMonth                  575 non-null    int64  
 5   period                    575 non-null    int64  
 6   reporterCode              575 non-null    int64  
 7   Country_ISO               575 non-null    object 
 8   Country                   575 non-null    object 
 9   flowCode                  575 non-null    object 
 10  flowDesc                  575 non-null    object 
 11  partnerCode               575 non-null    int64  
 12  partnerISO                575 non-null    object 
 13  partnerDesc               575 non-null    object 
 14  partner2Co

In [ ]:

import plotly.express as px

# Visualization: Faceted Plotly bar chart ---
fig = px.bar(
    df,
    x='Country',
    y='TradeValue_USD',
    color='Country',                
    facet_col='Year',               
    facet_col_wrap=3,               
    title="Oil Trade by Country (2019-2024)"
)

# Improves readability by removing "Year=" from the subplot titles
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig.show()




In [ ]:

# Filter the data for the specific years
gold_filtered = df[(df['Year'] >= 2019) & (df['Year'] <= 2024)]

# Sort the years to ensure they appear in order
years = sorted(gold_filtered['Year'].unique())

# Loop through each year and create a separate plot
for year in years:
    # Filter the dataframe for the specific year
    year_data = gold_filtered[gold_filtered['Year'] == year].sort_values(by='TradeValue_USD', ascending=False)

    # Create the individual bar chart
    fig = px.bar(
        year_data,
        x='Country',
        y='TradeValue_USD',
        color='Country',
        title=f"Oil Trade by Country ({year})",
        labels={'TradeValue_USD': 'Trade Value (USD)', 'Country': 'Country'}
    )

    # Optional: Hide the legend if there are many countries to save space
    fig.update_layout(showlegend=False)

    # Show the plot
    fig.show()

In [5]:
for year in range(2019, 2025):
    # Filter for the year, sort by value, and take top 3
    top_year = (df[df['Year'] == year]
                .groupby('Country')['TradeValue_USD']
                .sum()
                .sort_values(ascending=False)
                .head(5))

    print(f"\n--- Top 5 in {year} ---")
    for rank, (country, value) in enumerate(top_year.items(), 1):
        print(f"{rank}. {country} (${value:,.0f})")


--- Top 5 in 2019 ---
1. Saudi Arabia ($166,896,805,818)
2. United Arab Emirates ($127,284,293,030)
3. Russian Federation ($122,228,578,653)
4. Canada ($68,104,060,281)
5. USA ($65,323,191,031)

--- Top 5 in 2020 ---
1. United Arab Emirates ($105,123,364,796)
2. Saudi Arabia ($100,382,507,627)
3. Russian Federation ($72,564,293,958)
4. USA ($50,285,962,130)
5. Canada ($47,574,336,106)

--- Top 5 in 2021 ---
1. Saudi Arabia ($161,020,161,386)
2. Russian Federation ($110,968,246,742)
3. United Arab Emirates ($99,039,151,474)
4. Canada ($81,939,749,591)
5. USA ($69,356,113,331)

--- Top 5 in 2022 ---
1. Saudi Arabia ($258,544,564,094)
2. United Arab Emirates ($154,476,104,446)
3. Canada ($120,391,288,495)
4. USA ($117,034,462,388)
5. Kuwait ($68,036,100,177)

--- Top 5 in 2023 ---
1. Saudi Arabia ($175,993,253,254)
2. United Arab Emirates ($162,767,463,925)
3. USA ($117,160,520,564)
4. Canada ($99,464,941,105)
5. Nigeria ($52,520,992,085)

--- Top 5 in 2024 ---
1. Saudi Arabia ($179,494,

In [ ]:
import pandas as pd
import plotly.express as px


# Identify the Top 5 countries by total expenditure
top_3_countries = (
    df.groupby('Country')['TradeValue_USD']
    .sum()
    .sort_values(ascending=False)
    .head(5)   # actually Top 5
    .index
    .tolist()
)

# Filter the dataframe
df_top3 = df[df['Country'].isin(top_3_countries)]

# PLOT (unchanged logic, fixed title)
fig = px.line(
    df_top3,
    x='Year',
    y='TradeValue_USD',
    color='Country',
    markers=True,
    title="Oil Expenditure Trend: Top 5 Countries (2019–2024)",
    labels={'TradeValue_USD': 'Expenditure (USD)', 'Year': 'Year'},
    template="plotly_white"
)

fig.update_layout(
    xaxis=dict(tickmode='linear'),
    hovermode="x unified"
)

fig.show()

# AGGREGATE YEARLY EXPENDITURE
yearly_expenditure = (
    df_top3
    .groupby(['Year', 'Country'], as_index=False)['TradeValue_USD']
    .sum()
)

# YEAR-WISE HIGHEST → LOWEST PRINT WITH GAPS
yearwise_rank_exp = (
    yearly_expenditure
    .fillna(0)
    .sort_values(['Year', 'TradeValue_USD'], ascending=[True, False])
)

# Clean numbers for display
yearwise_rank_exp['TradeValue_USD'] = (
    yearwise_rank_exp['TradeValue_USD']
    .round(0)
    .astype(int)
)

print("\nYEAR-WISE OIL EXPENDITURE (USD)")
print("Countries ranked from Highest → Lowest\n")

for year, group in yearwise_rank_exp.groupby('Year'):
    print(f"YEAR {year}")
    print("-" * 55)

    group = group.reset_index(drop=True)
    group.index += 1

    print(
        group[['Country', 'TradeValue_USD']]
        .rename_axis('Rank')
        .reset_index()
        .to_string(index=False)
    )

    print("\n")



YEAR-WISE OIL EXPENDITURE (USD)
Countries ranked from Highest → Lowest

YEAR 2019
-------------------------------------------------------
 Rank              Country  TradeValue_USD
    1         Saudi Arabia    166896805818
    2 United Arab Emirates    127284293030
    3   Russian Federation    122228578653
    4               Canada     68104060281
    5                  USA     65323191031


YEAR 2020
-------------------------------------------------------
 Rank              Country  TradeValue_USD
    1 United Arab Emirates    105123364796
    2         Saudi Arabia    100382507627
    3   Russian Federation     72564293958
    4                  USA     50285962130
    5               Canada     47574336106


YEAR 2021
-------------------------------------------------------
 Rank              Country  TradeValue_USD
    1         Saudi Arabia    161020161386
    2   Russian Federation    110968246742
    3 United Arab Emirates     99039151474
    4               Canada     819397

In [ ]:
import pandas as pd
import plotly.express as px

# Define the Groups
BRICS = ["Brazil", "Russian Federation", "India", "China", "South Africa"]
US = ["USA"]
EU = ["Austria","Belgium","Bulgaria","Croatia","Cyprus","Czechia","Denmark",
      "Estonia","Finland","France","Germany","Greece","Hungary","Ireland","Italy",
      "Latvia","Lithuania","Luxembourg","Malta","Netherlands","Poland","Portugal",
      "Romania","Slovakia","Slovenia","Spain","Sweden"]

# Assign Group
def assign_group(country):
    if country in BRICS:
        return 'BRICS'
    if country in US:
        return 'US'
    if country in EU:
        return 'EU'
    return 'Other'

df['Group'] = df['Country'].apply(assign_group)

# Filter & Aggregate by Year and Group
df_groups = df[df['Group'].isin(['BRICS', 'US', 'EU'])]

group_summary = (
    df_groups
    .groupby(['Year', 'Group'], as_index=False)['TradeValue_USD']
    .sum()
)

# YEAR-WISE HIGHEST → LOWEST PRINT WITH GAPS
yearwise_group_rank = (
    group_summary
    .fillna(0)
    .sort_values(['Year', 'TradeValue_USD'], ascending=[True, False])
)

# Clean numbers
yearwise_group_rank['TradeValue_USD'] = (
    yearwise_group_rank['TradeValue_USD']
    .round(0)
    .astype(int)
)

print("\nYEAR-WISE GOLD EXPENDITURE (USD): BRICS vs US vs EU")
print("Groups ranked from Highest → Lowest\n")

for year, group in yearwise_group_rank.groupby('Year'):
    print(f"YEAR {year}")
    print("-" * 55)

    group = group.reset_index(drop=True)
    group.index += 1

    print(
        group[['Group', 'TradeValue_USD']]
        .rename_axis('Rank')
        .reset_index()
        .to_string(index=False)
    )

    print("\n") 

# PLOT (unchanged logic)
fig = px.line(
    group_summary,
    x='Year',
    y='TradeValue_USD',
    color='Group',
    markers=True,
    title="Gold Expenditure Trend: BRICS vs US vs EU (2019–2024)",
    labels={'TradeValue_USD': 'Total Expenditure (USD)', 'Year': 'Year'},
    template="plotly_white",
    color_discrete_map={
        'BRICS': '#E41A1C',
        'US': '#377EB8',
        'EU': '#4DAF4A'
    }
)

fig.update_layout(
    xaxis=dict(tickmode='linear'),
    hovermode="x unified"
)

fig.show()



YEAR-WISE GOLD EXPENDITURE (USD): BRICS vs US vs EU
Groups ranked from Highest → Lowest

YEAR 2019
-------------------------------------------------------
 Rank Group  TradeValue_USD
    1 BRICS    146790670740
    2    US     65323191031
    3    EU      2523009273


YEAR 2020
-------------------------------------------------------
 Rank Group  TradeValue_USD
    1 BRICS     92739479168
    2    US     50285962130
    3    EU      1938424543


YEAR 2021
-------------------------------------------------------
 Rank Group  TradeValue_USD
    1 BRICS    144066121607
    2    US     69356113331
    3    EU      3511893134


YEAR 2022
-------------------------------------------------------
 Rank Group  TradeValue_USD
    1    US    117034462388
    2 BRICS     44138723048
    3    EU     11320459914


YEAR 2023
-------------------------------------------------------
 Rank Group  TradeValue_USD
    1    US    117160520564
    2 BRICS     43455236886
    3    EU     20233006438


YEAR 2024


In [19]:
country_net = (
    df
    .groupby("Country")["netWgt"]
    .sum()
    .sort_values(ascending=False)
)

print(country_net.head(10))
country_net.head(10)
# Filter for the years 2019 to 2024
df_filtered = df[(df['Year'] >= 2019) & (df['Year'] <= 2024)]

# Aggregate the netWgt by Country
country_net = df_filtered.groupby('Country')['netWgt'].sum().sort_values(ascending=False)

# Get the top 10 countries
top_10_df = country_net.head(10).reset_index()

# Create the bar chart
fig = px.bar(
    top_10_df,
    x="Country",
    y="netWgt",
    color="Country",
    title="Top 10 Countries by Total Net Weight (2019 - 2024)",
    labels={
        "netWgt": "Total Net Weight (kg)",
        "Country": "Country"
    }
)

# Display the plot
fig.show()

Country
Saudi Arabia            1.975809e+12
USA                     1.180090e+12
Canada                  1.023951e+12
Russian Federation      7.399308e+11
Norway                  4.635563e+11
Kazakhstan              4.131921e+11
Angola                  3.347290e+11
Brazil                  3.086452e+11
Mexico                  2.838563e+11
United Arab Emirates    2.717302e+11
Name: netWgt, dtype: float64


In [20]:

# Filter for the years 2019 to 2024
df_filtered = df[(df['Year'] == 2024)]

# Aggregate the netWgt by Country
country_net = df_filtered.groupby('Country')['netWgt'].sum().sort_values(ascending=False)

# Get the top 10 countries
top_10_df = country_net.head(10).reset_index()
print(top_10_df)

# Create the bar chart
fig = px.bar(
    top_10_df,
    x="Country",
    y="netWgt",
    color="Country",
    title="Top 10 Countries by Total Net Weight (2024)",
    labels={
        "netWgt": "Total Net Weight (kg)",
        "Country": "Country"
    }
)

# Display the plot
fig.show()

        Country        netWgt
0  Saudi Arabia  3.005580e+11
1           USA  2.387884e+11
2        Canada  1.871811e+11
3        Brazil  8.966087e+10
4        Norway  8.402518e+10
5    Kazakhstan  7.102276e+10
6        Mexico  3.911451e+10
7   Netherlands  2.830895e+10
8      Colombia  2.543327e+10
9         Qatar  2.524238e+10


In [21]:
import plotly.express as px

# Group by Year and Country, sum netWgt
country_net = (
    df
    .groupby(['Year', 'Country'])['netWgt']
    .sum()
    .reset_index()
)

# List of years in your dataset
years = sorted(df['Year'].unique())

# Loop through each year and plot top 10 countries
for year in years:
    top_10_df = country_net[country_net['Year'] == year].sort_values(
        by='netWgt', ascending=False
    ).head(10)

    fig = px.bar(
        top_10_df,
        x="Country",
        y="netWgt",
        color="Country",
        title=f"Top 10 Countries by Net OIL in {year}",
        labels={
            "netWgt": "Net OIL (kg)",
            "Country": "Country"
        }
    )
    fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

# Group by Country across all years to get total NetWeight
country_net = df.groupby("Country")["netWgt"].sum().sort_values(ascending=False)

# Get top 8 countries
top_8_countries = country_net.index[:8].tolist()   # Top 8

# Filter data for these countries and sum by Year
trend_top8 = (
    df[df["Country"].isin(top_8_countries)]
    .groupby(["Year", "Country"], as_index=False)["netWgt"]
    .sum()
)

# PLOT
fig = px.line(
    trend_top8,
    x="Year",
    y="netWgt",
    color="Country",
    markers=True,
    title="Oil Reserve Trends for Top 8 Countries",
    labels={"netWgt": "Net Oil Change (kg)", "Year": "Year"}
)

fig.update_layout(
    xaxis=dict(tickmode="linear"),
    hovermode="x unified"
)

fig.show()

# YEAR-WISE HIGHEST → LOWEST PRINT
yearwise_rank = trend_top8.sort_values(["Year", "netWgt"], ascending=[True, False])

# Round numbers for clean display
yearwise_rank["netWgt"] = yearwise_rank["netWgt"].round(0).astype(int)

print("\nYEAR-WISE OIL NET WEIGHT (kg)")
print("Countries ranked from Highest → Lowest\n")

for year, group in yearwise_rank.groupby("Year"):
    print(f"YEAR {year}")
    print("-" * 45)

    group = group.reset_index(drop=True)
    group.index += 1  # Rank starting at 1

    print(
        group[["Country", "netWgt"]]
        .rename_axis("Rank")
        .reset_index()
        .to_string(index=False)
    )

    print("\n")  # gap between years



YEAR-WISE OIL NET WEIGHT (kg)
Countries ranked from Highest → Lowest

YEAR 2019
---------------------------------------------
 Rank            Country       netWgt
    1       Saudi Arabia 356301549773
    2 Russian Federation 269175244771
    3             Canada 145392729988
    4                USA 139455959546
    5             Angola  76402028259
    6         Kazakhstan  70008740152
    7             Norway  61278209950
    8             Brazil            0


YEAR 2020
---------------------------------------------
 Rank            Country       netWgt
    1       Saudi Arabia 335109400000
    2 Russian Federation 239170421480
    3                USA 184211339079
    4             Canada 156876691872
    5             Norway  75958060280
    6             Angola  70976644569
    7         Kazakhstan  70584352368
    8             Brazil  70009939004


YEAR 2021
---------------------------------------------
 Rank            Country       netWgt
    1       Saudi Arabia 3094275000

In [ ]:
# Define the Groups
BRICS = ["Brazil", "Russian Federation", "India", "China", "South Africa"]
US = ["USA"]
EU = ["Austria","Belgium","Bulgaria","Croatia","Cyprus","Czechia","Denmark",
      "Estonia","Finland","France","Germany","Greece","Hungary","Ireland","Italy",
      "Latvia","Lithuania","Luxembourg","Malta","Netherlands","Poland","Portugal",
      "Romania","Slovakia","Slovenia","Spain","Sweden"]

# Assign group
def assign_group(country):
    if country in BRICS: return 'BRICS'
    if country in US: return 'US'
    if country in EU: return 'EU'
    return 'Other'

df['Group'] = df['Country'].apply(assign_group)

# Filter and aggregate
df_groups = df[df['Group'].isin(['BRICS', 'US', 'EU'])]

group_summary = (
    df_groups
    .groupby(['Year', 'Group'])['netWgt']
    .sum()
    .reset_index()
)

# Print year-wise net df change
for year in sorted(group_summary['Year'].unique()):
    print(f"\n--- {year} Net Oil Change ---")
    year_data = group_summary[group_summary['Year'] == year] \
        .sort_values(by='netWgt', ascending=False)
    for _, row in year_data.iterrows():
        print(f"{row['Group']}: {row['netWgt']:,.0f} kg")

# Plot
fig = px.line(
    group_summary,
    x='Year',
    y='netWgt',
    color='Group',
    markers=True,
    title="Net Oil Reserve Change: BRICS vs US vs EU (2019–2024)",
    labels={'netWgt': 'Net Oil Change', 'Year': 'Year'},
    template="plotly_white",
    color_discrete_map={'BRICS': '#E41A1C', 'US': '#377EB8', 'EU': '#4DAF4A'}
)

fig.update_layout(xaxis=dict(tickmode='linear'), hovermode="x unified")
fig.show()



--- 2019 Net Oil Change ---
BRICS: 269,985,652,744 kg
US: 139,455,959,546 kg
EU: 3,460,953,967 kg

--- 2020 Net Oil Change ---
BRICS: 310,818,468,384 kg
US: 184,211,339,079 kg
EU: 4,854,360,733 kg

--- 2021 Net Oil Change ---
BRICS: 305,216,587,392 kg
US: 172,772,127,009 kg
EU: 5,930,982,343 kg

--- 2022 Net Oil Change ---
US: 209,184,635,442 kg
BRICS: 2,052,010,500 kg
EU: 1,105,873,919 kg

--- 2023 Net Oil Change ---
US: 235,677,354,408 kg
BRICS: 82,830,421,697 kg
EU: 1,415,187,618 kg

--- 2024 Net Oil Change ---
US: 238,788,419,322 kg
BRICS: 91,749,998,886 kg
EU: 31,376,794,650 kg


In [ ]:

# Filter for relevant years and valid data
df_filtered = df[(df['Year'] >= 2019) & (df['Year'] <= 2024)]
oil_filtered = df_filtered[(df_filtered['netWgt'] > 0) & (df_filtered['TradeValue_USD'] > 0)]

# Aggregate Data by Year and Flow
trend_data = oil_filtered.groupby(['Year', 'flowDesc']).agg({
    'TradeValue_USD': 'sum',
    'netWgt': 'sum'
}).reset_index()

# Calculate Price per Barrel (Converting kg to Barrel)
# Standard conversion: 1 Barrel of Crude Oil ≈ 136.4 kg
CONVERSION_FACTOR = 136.4
trend_data['Price_per_Barrel'] = (trend_data['TradeValue_USD'] / trend_data['netWgt']) * CONVERSION_FACTOR

# --- NEW: Print the calculated values ---
print("Oil Price Trend Data (2019-2024):")
print(trend_data[['Year', 'flowDesc', 'TradeValue_USD', 'Price_per_Barrel']].to_string(index=False))
print("-" * 30)

# Create Plotly Line Chart
fig = px.line(
    trend_data,
    x='Year',
    y='Price_per_Barrel',
    color='flowDesc',
    title='Global Oil Price Trend per Barrel (2019 - 2024)',
    labels={
        'Price_per_Barrel': 'Price (USD per Barrel)',
        'Year': 'Year',
        'flowDesc': 'Trade Flow'
    },
    markers=True,
    template='plotly_white'
)

# Customize the layout for better readability
fig.update_layout(
    hovermode='x unified',
    yaxis_tickprefix='$',
    yaxis_tickformat='.2f'
)

fig.show()

Oil Price Trend Data (2019-2024):
 Year flowDesc  TradeValue_USD  Price_per_Barrel
 2019   Export    7.896565e+11         63.146065
 2020   Export    4.592606e+11         40.289746
 2021   Export    6.902119e+11         65.224982
 2022   Export    9.171721e+11         91.722694
 2023   Export    7.529181e+11         74.339768
 2024   Export    7.024241e+11         80.416219
------------------------------


In [ ]:

# Define the Groups
BRICS = ["Brazil", "Russian Federation", "India", "China", "South Africa"]
US = ["USA"]
EU = ["Austria","Belgium","Bulgaria","Croatia","Cyprus","Czechia","Denmark",
      "Estonia","Finland","France","Germany","Greece","Hungary","Ireland","Italy",
      "Latvia","Lithuania","Luxembourg","Malta","Netherlands","Poland","Portugal",
      "Romania","Slovakia","Slovenia","Spain","Sweden"]

# Create a Group mapping column
def categorize(country):
    if country in BRICS: return 'BRICS'
    if country in US: return 'US'
    if country in EU: return 'EU'
    return 'Other'

df['Category'] = df['Country'].apply(categorize)

# Filter for years since 2021 and only the groups of interest
# Using 'netWgt' (kg) as the volume metric
df_trends = df[(df['Year'] >= 2021) & (df['Category'] != 'Other')]
df_yearly = df_trends.groupby(['Year', 'Category'])['netWgt'].sum().reset_index()

# Calculate CAGR (2021 to 2024)
start_year = 2021
end_year = df_yearly['Year'].max()
years_diff = end_year - start_year

cagr_data = []
for cat in df_yearly['Category'].unique():
    subset = df_yearly[df_yearly['Category'] == cat]
    try:
        v_start = subset[subset['Year'] == start_year]['netWgt'].values[0]
        v_end = subset[subset['Year'] == end_year]['netWgt'].values[0]
        if v_start > 0:
            cagr = (v_end / v_start)**(1 / years_diff) - 1
            cagr_data.append({'Category': cat, 'CAGR': f"{cagr:.2%}"})
        else:
            cagr_data.append({'Category': cat, 'CAGR': "Invalid Base"})
    except IndexError:
        cagr_data.append({'Category': cat, 'CAGR': "Incomplete Data"})

print("--- CAGR Results (2021-2024) ---")
print(pd.DataFrame(cagr_data))

# Plot the Trend using Plotly Express
fig = px.line(
    df_yearly,
    x='Year',
    y='netWgt',
    color='Category',
    title='Oil Trade Volume Trend by Group (2021 - 2024)',
    labels={'netWgt': 'Total Weight (kg)', 'Year': 'Year'},
    markers=True,
    template='plotly_white'
)

fig.update_layout(hovermode='x unified')
fig.show()

--- CAGR Results (2021-2024) ---
  Category     CAGR
0    BRICS  -33.01%
1       EU   74.25%
2       US   11.39%
